<a href="https://colab.research.google.com/github/PaoHdez94/hacking-civico/blob/master/tareas/03_exploracion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COVID-19 en México

Carga

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
import io
import statistics
import pandas as pd
import numpy as np

path = '/content/drive/My Drive/EJEMPLO COVID/200908COVID19MEXICO.csv'
data = pd.read_csv(path,encoding='latin-1')

data.head()

In [ ]:
path_dic = '/content/drive/My Drive/EJEMPLO COVID/diccionario_datos_covid19/Descriptores_0419.xlsx'
diccionario = pd.read_excel(path_dic)

diccionario

In [ ]:
path_cat = '/content/drive/My Drive/EJEMPLO COVID/diccionario_datos_covid19/Catalogos_0412.xlsx'
sheets = ['ORIGEN','SECTOR','SEXO','TIPO_PACIENTE','SI_NO','NACIONALIDAD','RESULTADO','de ENTIDADES','MUNICIPIOS']
catalogues = {}
for sheet in sheets:
  catalogue = pd.read_excel(path_cat, sheet_name='Catálogo ' + sheet)
  catalogue.columns = catalogue.columns.str.replace(' ', '_')
  catalogue.columns = catalogue.columns.str.replace('.', '')
  catalogue.columns = catalogue.columns.str.lower()
  new_catalogue = {sheet: catalogue}
  catalogues.update(new_catalogue)

catalogues

In [72]:
catalogues['TIPO_PACIENTE']

,clave,descripción
0,1,AMBULATORIO
1,2,HOSPITALIZADO
2,99,NO ESPECIFICADO


# PERFILAMIENTO
## Descripición General

In [ ]:
# Dimensión de los datos
print('Número de filas: {}'.format(data.shape[0]))
print('Número de columnas: {}'.format(data.shape[1]))

In [ ]:
# Columnas (variables)
data.columns

In [58]:
# Limpieza del nombre de las columnas, remover espacios, carácteres especiales y pasar a minúsculas
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace('.', '')
data.columns = data.columns.str.lower()

In [ ]:
# Calcular número de valores nulos por columna
for col in data.columns:
  print('Valores nulos en "{}": {}'.format(col,data[col].isna().sum()))

In [ ]:
# Tipos de datos
data.dtypes

In [49]:
# Convertimos a fecha algunas columnas
def cast_datetime_cols(df):
    timestamp_cols = [col for col in df.columns if "fecha" in col]
    df[timestamp_cols] = df[timestamp_cols].apply(lambda date_col: pd.to_datetime(date_col, errors="coerce"), axis=0)
    return df

data = cast_datetime_cols(data)

In [ ]:
data.head()

In [ ]:
# Unimos los datos con los catalogos para tener mejor entendimiento de ellos
data = (
    # Unimos con el catálogo MUNICIPIOS y ENTIDADES
    data.merge(catalogues['de ENTIDADES'], how='left', left_on='entidad_res', right_on='clave_entidad').
    drop(columns=['entidad_res', 'abreviatura']).
    rename(columns={'entidad_federativa':'entidad_res'}).
    merge(catalogues['MUNICIPIOS'], how='left', left_on=['clave_entidad','municipio_res'], right_on=['clave_entidad','clave_municipio']).
    drop(columns=['municipio_res', 'clave_entidad', 'clave_municipio']).
    rename(columns={'municipio':'municipio_res'})
)

In [ ]:
# Unimos con el catálogo de ENTIDADES
entidades_cols = [col for col in data.columns if "entidad" in col and col != 'entidad_res']
for col in entidades_cols:
    data = (
        data.merge(catalogues['de ENTIDADES'], how='left', left_on=col, right_on='clave_entidad').
        drop(columns=[col, 'abreviatura', 'clave_entidad']).
        rename(columns={'entidad_federativa':col})
    )

In [ ]:
# Unimos con el catálogo SI_NO
sino_cols = ['intubado', 'neumonia', 'embarazo', 'habla_lengua_indig', 'diabetes', 'epoc', 'asma', 'inmusupr',
             'hipertension', 'otra_com', 'cardiovascular', 'obesidad', 'renal_cronica', 'tabaquismo', 
             'otro_caso', 'migrante', 'uci']
for col in sino_cols:
    data = (
        data.merge(catalogues['SI_NO'], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [ ]:
# Corregimos nombre de columnas
catalogues['RESULTADO'].columns

In [ ]:
catalogues['RESULTADO'].columns = ['clave', 'descripción']
catalogues['RESULTADO'].columns

In [ ]:
# Removemos la primer fila que no es útil
catalogues['RESULTADO'] = catalogues['RESULTADO'].drop(index=0)

In [40]:
# Convertimos el tipo de dato
catalogues['RESULTADO']['clave'] = catalogues['RESULTADO']['clave'].astype(int)

In [ ]:
# Fin dela limpieza
# Unimos otras columnnas que comparten lógica
other_cols = ['origen', 'sector', 'sexo', 'tipo_paciente', 'nacionalidad', 'resultado']
for col, cat in zip(other_cols, [x.upper() for x in other_cols]):
    data = (
        data.merge(catalogues[cat], how='left', left_on=col, right_on='clave').
        drop(columns=[col, 'clave']).
        rename(columns={'descripción':col})
    )

In [45]:
# Creamos algunas variables que podrían ser útiles después
data['defuncion'] = data.fecha_def.isna()
data['mes_def'] = data.fecha_def.dt.month
data['año_def'] = data.fecha_def.dt.year
data['mes_ingreso'] = data.fecha_ingreso.dt.month
data['año_ingreso'] = data.fecha_ingreso.dt.year

In [ ]:
data.head()

# ¿Tienen los pacientes con hipertensión un riesgo más alto de defunción?

In [67]:
# Hipertensión
data.groupby('hipertension').defuncion.value_counts(normalize=True)

hipertension  defuncion
1             True         0.833960
              False        0.166040
2             True         0.960276
              False        0.039724
98            True         0.874874
              False        0.125126
Name: defuncion, dtype: float64

# ¿Cuántos casos confirmados se tienen por Estado?

In [69]:
# Valores por Entidad
data.entidad_res.value_counts()

CIUDAD DE MÉXICO                   297270
MÉXICO                             178629
GUANAJUATO                          82439
NUEVO LEÓN                          74491
PUEBLA                              61427
TAMAULIPAS                          59580
TABASCO                             58103
JALISCO                             56616
COAHUILA DE ZARAGOZA                56397
VERACRUZ DE IGNACIO DE LA LLAVE     47492
SAN LUIS POTOSÍ                     43722
MICHOACÁN DE OCAMPO                 42085
SONORA                              41707
SINALOA                             30521
YUCATÁN                             30267
BAJA CALIFORNIA                     29084
GUERRERO                            27829
OAXACA                              21421
DURANGO                             21290
CHIHUAHUA                           20209
HIDALGO                             19265
BAJA CALIFORNIA SUR                 19054
AGUASCALIENTES                      18775
TLAXCALA                          

# ¿Cuántas defunciones se tienen por Estado?

In [79]:
lista = data.groupby(['defuncion'])['entidad_res'].value_counts()
lista.head(60)

defuncion  entidad_res                    
False      MÉXICO                              13264
           CIUDAD DE MÉXICO                    11807
           VERACRUZ DE IGNACIO DE LA LLAVE      4727
           PUEBLA                               4524
           BAJA CALIFORNIA                      4019
           JALISCO                              3751
           SINALOA                              3651
           NUEVO LEÓN                           3392
           SONORA                               3232
           GUANAJUATO                           2918
           TABASCO                              2909
           TAMAULIPAS                           2779
           GUERRERO                             2232
           COAHUILA DE ZARAGOZA                 2052
           HIDALGO                              2009
           CHIHUAHUA                            1991
           QUINTANA ROO                         1866
           MICHOACÁN DE OCAMPO                  1826
   

# ¿Cuántos fallecimientos han ocurrido en los pacientes Ambulatorios?

In [78]:
data.groupby ('defuncion').tipo_paciente.value_counts()

defuncion  tipo_paciente
False      HOSPITALIZADO      78141
           AMBULATORIO         9327
True       AMBULATORIO      1185652
           HOSPITALIZADO     177277
Name: tipo_paciente, dtype: int64